In [158]:
"""ATTENTION:

This notebook relies on the cleaned english data after running clean_english_dataset.py. The only prerequisite is that the data doesn't have invalid or non-english tokens.
"""
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import smart_open, simple_preprocess, lemmatize
import math
import pandas as pd
import nltk
words = set(nltk.corpus.words.words())

In [159]:
data = pd.read_csv('stories_content_english_corrected.csv')
publisher_stories = data.loc[data['story_publishing_type'] == 'Publisher Story']
#print(publisher_stories['story_title'])

0                                         Smile Please!
1                                     Fat King Thin Dog
2                                        Vayu, the Wind
3                               Chuskit Goes to School!
4                                     Not Now, Not Now!
5                                      Counting on Moru
6                                     Listen to My Body
7                            "My fish!"  "No, my fish!"
8                                     Susheela's Kolams
9                                          Pehelwaan Ji
10      Sister, Sister, Where Does the Sun Go at Night?
11                                Everything looks new!
12                                     Aunty Jui's Baby
13                                     I Want That One!
14                                Hot Tea and Warm Rugs
15                           Kheer on a Full Moon Night
16                                Peacocks and Pakodas!
17                                     Goloo the

In [160]:
print(len(publisher_stories))
tokens_data = [simple_preprocess(d) for d in data['story_content']]
# print(tokens_data)
for i in range(len(tokens_data)):
    tokens_data[i] = [lemmatize(t) for t in tokens_data[i] if t not in STOPWORDS and t in words]

311


In [168]:
tokens_data[:10]
# print([word for sublist in tokens_data for word in sublist][:5])
# type(tokens_data)
doc_tokens = []
for sublist in tokens_data:
    doc_tokens.append([word for inner_list in sublist for word in inner_list])
#print(doc_tokens[:10])
id2word = Dictionary(doc_tokens)

[[b'fawn/NN', b'race/VB', b'forest/NN', b'ahead/RB', b'rabbit/NN', b'ahead/RB', b'elephant/NN', b'leap/VB', b'stream/NN', b'run/VB', b'past/JJ', b'wall/NN', b'large/JJ', b'boulder/NN', b'grassy/JJ', b'plain/RB', b'fall/VB', b'burst/RB', b'monkey/NN', b'leg/NN', b'fawn/NN', b'brother/NN', b'bear/NN', b'picked/JJ', b'fawn/NN', b'stop/VB', b'cry/VB', b'mother/NN', b'come/VB', b'say/VB', b'look/VB', b'beat/JJ', b'bad/JJ', b'boulder/NN', b'fawn/NN', b'say/VB', b'start/NN', b'cry/VB', b'mother/NN', b'fawn/NN'], [b'fat/JJ', b'king/NN', b'fat/JJ', b'king/NN', b'dog/NN', b'fat/JJ', b'king/NN', b'dog/NN', b'walk/NN', b'dog/NN', b'bird/NN', b'dog/NN', b'bird/NN', b'king/NN', b'dog/NN', b'run/VB', b'run/VB', b'run/VB', b'run/VB', b'day/NN', b'king/NN', b'dog/NN', b'fat/JJ', b'king/NN'], [b'finish/NN', b'hot/JJ', b'hot/JJ', b'bath/NN', b'wet/JJ', b'body/NN', b'cool/JJ', b'cool/JJ', b'cool/JJ', b'happen/VB', b'wind/NN', b'milk/NN', b'cup/NN', b'hot/JJ', b'hot/JJ', b'soon/RB', b'ready/JJ', b'gulp/NN'

In [170]:
# for i in range(0, 100):
#     print(id2word.get(i))

ahead/RB
bad/JJ
bear/NN
beat/JJ
boulder/NN
brother/NN
burst/RB
come/VB
cry/VB
elephant/NN
fall/VB
fawn/NN
forest/NN
grassy/JJ
large/JJ
leap/VB
leg/NN
look/VB
monkey/NN
mother/NN
past/JJ
picked/JJ
plain/RB
rabbit/NN
race/VB
run/VB
say/VB
start/NN
stop/VB
stream/NN
wall/NN
bird/NN
day/NN
dog/NN
fat/JJ
king/NN
walk/NN
away/RB
bath/NN
black/JJ
body/NN
bolt/NN
brush/NN
cool/JJ
course/NN
cup/NN
din/NN
face/NN
fall/NN
far/RB
finish/NN
flutter/NN
gently/RB
glass/NN
god/NN
ground/NN
gulp/NN
happen/VB
hear/VB
hot/JJ
house/NN
leave/VB
lightning/NN
milk/NN
mischief/NN
move/VB
ready/JJ
see/VB
sill/NN
smell/RB
soon/RB
sway/RB
thank/NN
train/NN
tremble/VB
tumbler/NN
way/NN
wet/JJ
whistle/NN
wind/NN
window/NN
word/NN
work/NN
aba/RB
able/JJ
accord/VB
ama/NN
angrily/RB
announce/VB
apricot/NN
arm/NN
arrival/NN
ask/NN
assembly/NN
awake/VB
backward/RB
bag/NN
bed/NN
begin/VB
believe/VB


In [172]:
bow = [id2word.doc2bow(doc) for doc in doc_tokens]

In [173]:
tag_data = pd.read_csv('stories_tags.csv')

In [174]:
tag_dict = {}
tag_set = set()
tag_count = {}
for title in publisher_stories['story_title']:
    indices = [i for i,x in enumerate(tag_data['story_title']) if x == title]
    tag_list = [tag_data['story_tag_name'].iloc[i] for i in indices]
    for tag in tag_list:
        if type(tag) == str and len(simple_preprocess(tag)) > 1.0:
            smaller_tags = simple_preprocess(tag)
            tag_list.remove(tag)
            tag_list.append(smaller_tags)
    tag_list = [tag.replace('#','').lower() for tag in tag_list if (type(tag) == str and tag not in STOPWORDS)]
    tag_list = [tag for tag in tag_list if tag in words]
    if len(tag_list) <= 1:
        continue
#     print(title, tag_list)
    tag_dict[title] = tag_list
    for tag in tag_list:
        if tag not in tag_count:
            tag_count[tag] = 1
        else:
            tag_count[tag] += 1
    tag_set |= set(tag_list)
    
usable_stories = len(tag_dict)
# print(tag_set)
print(len(tag_set))
print(len([tag for tag in tag_count if tag_count[tag] >= 5]))

estimated_topics = len([tag for tag in tag_count if tag_count[tag] >= 5])

700
73


In [175]:
ldamodel5 = LdaModel(bow, num_topics=estimated_topics, id2word=id2word, passes=5)
ldamodel10 = LdaModel(bow, num_topics=estimated_topics, id2word=id2word, passes=10)
ldamodel20 = LdaModel(bow, num_topics=estimated_topics, id2word=id2word, passes=20)

In [176]:
for i in ldamodel5.print_topics():
    print(i)
    print()
    
print("-------------------------------------------------")

for i in ldamodel10.print_topics():
    print(i)
    print()
    
print("-------------------------------------------------")

for i in ldamodel20.print_topics():
    print(i)
    print()
    
print("-------------------------------------------------")



(45, '0.148*"economic/JJ" + 0.085*"dump/VB" + 0.077*"effect/NN" + 0.075*"state/VB" + 0.066*"wake/NN" + 0.028*"come/VB" + 0.027*"medical/JJ" + 0.021*"jordan/RB" + 0.020*"morning/NN" + 0.016*"need/VB"')

(60, '0.050*"tree/NN" + 0.021*"mango/NN" + 0.019*"grow/VB" + 0.019*"fox/NN" + 0.017*"night/NN" + 0.017*"seed/NN" + 0.016*"kite/NN" + 0.014*"fruit/NN" + 0.013*"garden/NN" + 0.011*"day/NN"')

(13, '0.051*"tree/NN" + 0.050*"see/VB" + 0.047*"say/VB" + 0.037*"go/VB" + 0.035*"come/VB" + 0.034*"dead/JJ" + 0.033*"ahead/RB" + 0.033*"man/NN" + 0.023*"hang/VB" + 0.020*"appearance/NN"')

(20, '0.048*"bidding/NN" + 0.032*"tum/NN" + 0.030*"ti/NN" + 0.029*"tee/NN" + 0.025*"la/VB" + 0.022*"boy/NN" + 0.020*"horse/NN" + 0.017*"give/VB" + 0.014*"buy/VB" + 0.013*"look/VB"')

(40, '0.121*"water/NN" + 0.040*"drink/VB" + 0.031*"dirty/JJ" + 0.026*"river/NN" + 0.025*"come/VB" + 0.023*"idea/NN" + 0.021*"visit/NN" + 0.021*"day/NN" + 0.020*"tell/VB" + 0.018*"problem/NN"')

(62, '0.239*"lion/NN" + 0.033*"wind/NN" + 